In [15]:
import cv2
import numpy as np

In [16]:
# Criação de variaveis globais
arm_extended = False # Estado do braço (esticado ou nao)
eagle_active = False # Estado da aguia (ativa ou nao)
eagle_idle = False # Se esta a usar o video idle
eagle_cap = cv2.VideoCapture("videos/eagleLanding.mp4") # Video da aguia
idle_eagle_cap = cv2.VideoCapture("videos/eagleIdle.mp4") # Video da aguia idle
video0 = "videos/video0.avi"
ini = 0 # Frame inicial
fin = 260 # Frame final

In [17]:
def extrairPessoa(img):
    # Converter para grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Usar Otsu para binarização
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Operações morfológicas para limpar buracos na máscara
    kernel = np.ones((5, 5), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # Encontrar contornos
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Construir máscara a partir dos contornos
    mask = np.zeros(img.shape[:2], np.uint8)
    for cnt in contours:
        if cv2.contourArea(cnt) > 5000:  # ignorar contornos pequenos
            cv2.drawContours(mask, [cnt], -1, 255, thickness=-1)

    mask = cv2.bitwise_not(mask)  # Invert mask

    # Extrair a pessoa utilizando a máscara
    fg = cv2.bitwise_and(img, img, mask=mask)
    return fg, mask, contours

In [18]:
def cortarVideo(video0, ini, fin):
    original = cv2.VideoCapture(video0)
    if not original.isOpened():
        raise ValueError("Erro ao abrir o video")

    total = int(original.get(cv2.CAP_PROP_FRAME_COUNT))
    fps   = original.get(cv2.CAP_PROP_FPS)
    width = int(original.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(original.get(cv2.CAP_PROP_FRAME_HEIGHT))

    ini = max(0, ini)
    fin = min(fin, total - 1)
    if ini > fin:
        raise ValueError("ini tem de ser <= fin")

    out_name = "video1.avi"
    fourcc = cv2.VideoWriter_fourcc('X', 'V', 'I', 'D')
    writer = cv2.VideoWriter(out_name, fourcc, fps, (width, height), True)

    current = 0
    while original.isOpened():
        ret, frame = original.read()
        
        if not ret:
            break

        if ini <= current <= fin:
            writer.write(frame)

        if current > fin:
            break

        current += 1

    original.release()
    writer.release()
    return out_name



In [ ]:
def classificarMao(contours, centroid, img):
    global eagle_active, eagle_idle, arm_extended
    cnt = max(contours, key=cv2.contourArea)  # Maior contorno
    
    # print("Contour area:", cv2.contourArea(cnt))
    
    pts = cnt.reshape(-1, 2)  # Converter contorno para array de pontos 2D
    # Encontrar ponto mais distante do centroide (mão)
    dx = pts[:, 0] - centroid[0] # Lista das distancias de x ao centroide
    dy = pts[:, 1] - centroid[1] # Lista das distancias de y ao centroide
    dists = np.hypot(dx, dy) # Lista das distancias euclidianas ao centroide

    max_idx = np.argmax(dists) # Indice do ponto mais distante
    hand_x, hand_y = pts[max_idx] # Coordenadas do ponto mais distante (mão)
    hand_dist = dists[max_idx] # Distancia até ao ponto mais distante (mão)
    
    # print("Hand coordinates:", (hand_x, hand_y), "Distance from centroid:", hand_dist)
    
    x, y, w, h = cv2.boundingRect(cnt) # Obter bounding box do corpo

    # Condições para considerar "braço esticado"
    if hand_dist > 0.3 * w: # Se a distância da mão ao centroide for > 30% da largura do corpo
        # dy[max_idx], dx[max_idx] é o vetor centroide -> mão
        angle_deg = np.degrees(np.arctan2(dy[max_idx], dx[max_idx])) # Obter o angulo em graus
        # Em OpenCV, 0º é à direita, 90º é para baixo...
        
        # Para o brazo estar horizontal, o ângulo deve ser próximo de 0º ou 180º
        is_horizontal = (abs(angle_deg) < 30) or (abs(abs(angle_deg) - 180) < 30)

        if is_horizontal:
            # Vetores da mão ao centroide normalizados
            ux = dx[max_idx] / hand_dist
            uy = dy[max_idx] / hand_dist

            # Ponto "ideal" no meio do braço
            # Ponto da centroide + vetor mão * 0.5 * distância mão-centroide
            target_x = centroid[0] + ux * hand_dist * 0.5
            target_y = centroid[1] + uy * hand_dist * 0.5

            # Vetores de deslocamento até ao ponto ideal
            diff_tx = pts[:, 0] - target_x
            diff_ty = pts[:, 1] - target_y

            dist_to_target = np.hypot(diff_tx, diff_ty) # Lista das distancias euclidianas ao ponto ideal
            mid_idx = np.argmin(dist_to_target) # Obtem o ponto do contorno mais proximo ao ponto ideal
            arm_cx, arm_cy = pts[mid_idx] # Coordenadas desse ponto
            
            xs = pts[:, 0] # Todos os valores X do contorno
            diff_x = xs - arm_cx # Subtrair o X do braço de cada ponto do contorno
            abs_diff_x = np.abs(diff_x) # Tirar o valor absoluto das diferenças
            mask = abs_diff_x <= 5 # Mascara True para pontos na mesma coluna +-5px
            mesma_coluna = np.where(mask)[0] # Obter os indices dos pontos onde a condição é verdadeira

            if len(mesma_coluna) > 0: # Se houver
                sub_pts = pts[mesma_coluna] # Pontos que estão na coluna do braço
                top_idx_local = np.argmin(sub_pts[:, 1]) # Obtem o menor Y da lista de pontos
                arm_cx, arm_cy = sub_pts[top_idx_local] # Coordenadas finais

            arm_cx, arm_cy = int(arm_cx), int(arm_cy)
            cv2.circle(img, (arm_cx, arm_cy), 20, (255, 0, 0), -1)
            cv2.putText(img, "Braco esticado",
                        (arm_cx + 10, arm_cy - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 2,
                        (255, 0, 0), 5, cv2.LINE_AA)
            
            arm_extended = True
            # Ativar a aguia se ainda nao estiver ativa
            if not eagle_active:
                eagle_active = True
                eagle_idle = False
                eagle_cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            return (arm_cx, arm_cy)
        else:
            arm_extended = False # Se não entrar no if, braço não esta esticado

In [20]:
def ativarAguia(img, arm):
    global eagle_active, eagle_idle, arm_extended, eagle_cap, idle_eagle_cap
    # Se a aguia está ativa mas o braço já não está esticado, desativar a aguia e resetar o video
    if eagle_active and not arm_extended:
        eagle_active = False
        eagle_idle = False
        eagle_cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        idle_eagle_cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    # Desenhar aguia se estiver ativa
    if eagle_active:

        # Decidir que video dar play
        if eagle_idle:
            cap = idle_eagle_cap # Video idle em loop
        else:
            cap = eagle_cap # Video de aterragem
        ret_e, eagle_frame = cap.read() # Ler primeiro frame do video escolhido

        # Video atual acabou
        if not ret_e:
            if not eagle_idle: # Acabou o video da aterragem
                eagle_idle = True # Mudar para idle
                cap = idle_eagle_cap
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0) # Resetar idle
            else: # Acabou o video idle
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0) # Resetar idle

            # Ler novamente
            ret_e, eagle_frame = cap.read()
            if not ret_e:
                return 0 #ATENCAO
            
        if eagle_active and ret_e:
            # Redimensionar a aguia
            scale = 0.5
            eagle_frame = cv2.resize(eagle_frame, None, fx=scale, fy=scale)

            # Converter para HSV para facilitar detetar verde da greenscreen
            hsv = cv2.cvtColor(eagle_frame, cv2.COLOR_BGR2HSV)
            # Intervalo do verde
            lower_green = np.array([35, 40, 40])
            upper_green = np.array([85, 255, 255])
            # Mascara da aguia
            eagle_mask = cv2.inRange(hsv, lower_green, upper_green) # Apenas o verde
            eagle_mask_inv = cv2.bitwise_not(eagle_mask) # Apenas a aguia (sem verde)

            eh, ew = eagle_frame.shape[:2] # Altura e largura da aguia

            # Offsets para ajustar posição da aguia no braço
            xOS = -10
            yOS = 100
            # x0, y0 -> ponto superior esquerdo da zona onde vai ser desenhada a aguia
            x0 = arm[0] - ew // 2 + xOS # X inicial = braço - metade da largura da aguia + offset
            y0 = arm[1] - eh + yOS # Y inicial = braço - altura da aguia + offset

            # Clipping á esquerda
            if x0 < 0:
                cut = -x0 # Quantidade a cortar
                eagle_frame = eagle_frame[:, cut:] # Cortar colunas á esquerda
                eagle_mask = eagle_mask[:, cut:]
                eagle_mask_inv = eagle_mask_inv[:, cut:]
                x0 = 0 # Repor x0 no limite
            # Clipping em cima
            if y0 < 0:
                cut = -y0
                eagle_frame = eagle_frame[cut:, :] # Cortar linhas em cima
                eagle_mask = eagle_mask[cut:, :]
                eagle_mask_inv = eagle_mask_inv[cut:, :]
                y0 = 0
            # Recalcular dimensoes apos cortes
            eh, ew = eagle_frame.shape[:2]
            x1 = x0 + ew # x1 = x0 + largura da aguia
            y1 = y0 + eh # y1 = y0 + altura da aguia

            H, W = img.shape[:2] # Altura e largura da imagem da webcam
            # Clipping á direita
            if x1 > W:
                cut = x1 - W # Quantidade a cortar
                eagle_frame = eagle_frame[:, :-cut] # Cortar colunas á direita
                eagle_mask = eagle_mask[:, :-cut]
                eagle_mask_inv = eagle_mask_inv[:, :-cut]
                ew = eagle_frame.shape[1]
                x1 = W # x1 fica no limite direito
                x0 = x1 - ew # Ajustar x0 para manter largura
            # Clipping em baixo
            if y1 > H:
                cut = y1 - H
                eagle_frame = eagle_frame[:-cut, :] # Cortar linhas em baixo
                eagle_mask = eagle_mask[:-cut, :]
                eagle_mask_inv = eagle_mask_inv[:-cut, :]
                eh = eagle_frame.shape[0]
                y1 = H # y1 fica no limite inferior
                y0 = y1 - eh # Ajustar y0 para manter altura

            eagleZone = img[y0:y1, x0:x1] # Area da imagem onde a aguia sera desenhada

            eagleHole = cv2.bitwise_and(eagleZone, eagleZone, mask=eagle_mask) # Zona onde fica o fundo (buraco em forma de aguia)
            eagleOnly = cv2.bitwise_and(eagle_frame, eagle_frame, mask=eagle_mask_inv) # Apenas a aguia (sem verde)

            eagleFinal = cv2.add(eagleHole, eagleOnly) # Somar buraco e aguia
            img[y0:y1, x0:x1] = eagleFinal # Meter resultado na imagem
    

In [ ]:
def AugmentedReality(video0, ini, fin):
    if (ini > 0) or (fin > 0): # Cortar video se ini e fin forem diferentes de -1
        video1 = cortarVideo(video0 ,ini, fin)
    else:
        video1 = video0
    framerate = 60
    delayFrame = int(1.0 / framerate * 1000)
    
    vidBuffer = cv2.VideoCapture(video1) # Abrir video cortado ou original
    if (video0 != 0):
        cv2.namedWindow("Mask", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Mask", 500, 300)

        cv2.namedWindow("Final", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Final", 500, 300)

    while vidBuffer.isOpened():
        flag, img = vidBuffer.read() # Ler frame atual do video
        if not flag:
            break
        
        _, mask, _ = extrairPessoa(img) # Obter máscara da pessoa
        cv2.imshow("Mask", mask)

        M = cv2.moments(mask) # Calcular momentos da mascara
    
        # Calcular centroide
        cX = int(M["m10"] / M["m00"]) 
        cY = int(M["m01"] / M["m00"])
        
        cv2.circle(img, (cX, cY), 15, (255, 255, 255), -1) # Desenhar centroide
        cv2.putText(img, "centroid", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 5) # Desenhar texto do centroide
        
        contours2, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # Calcular contornos na mascara
        
        arm = classificarMao(contours2, (cX, cY), img)
        
        ativarAguia(img, arm)
        
        cv2.imshow("Final", img)
        
        key = cv2.waitKey(delayFrame)
        
        if key == ord('s'):  # Apertar S para salvar
            cv2.imwrite("frame_salvo.png", img)
        
        if key == 27:  # ESC para sair
            break

    vidBuffer.release()
    cv2.destroyAllWindows()
    
# Para utilizar webcam, colocar 0 como video0 e -1 para ini e fin
video1 = AugmentedReality(video0 ,ini, fin)

    